In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_absolute_error as mae
import pickle
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
#from neupy.algorithms import RBFKMeans
#from neupy.algorithms import GRNN
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn import svm
from pyGRNN import GRNN
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
import seaborn as sn
import joblib
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from skopt import BayesSearchCV
import warnings
import heapq
from sklearn.model_selection import RepeatedKFold
from scipy.stats import randint as sp_randint
from skopt.space import Real, Categorical, Integer

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
# Utility function to report best scores+timings
def report_perf(optimizer, X, y, title):
    optimizer.fit(X, y)
    print(title, "best CV score:", optimizer.best_score_)

In [ ]:
'''
Random forest (RF)
'''

file_name = 'NiTi_PhaseDatesets.xlsx'
dataset = pd.read_excel(file_name, header=0)
#print(dataset)
x1 = dataset['P(W)'].values
x2 = dataset['V(mm/s)'].values
x3 = dataset['h(um)'].values
# x4 = dataset['t(um)'].values
x5 = dataset['LED(J/mm)'].values
# x6 = dataset['SED(J/mm2)'].values
x7 = dataset['VED(J/mm3)'].values
x8 = dataset['Ni'].values
# x9 = dataset['Ti'].values
x10 = dataset['Dmin'].values
x11 = dataset['Dmax'].values
x12 = dataset['D10'].values
# x13 = dataset['D50'].values
x14 = dataset['D90'].values


y1 = dataset['Ms'].values
y2 = dataset['Mf'].values
y3 = dataset['As'].values
y4 = dataset['Af'].values
X = np.column_stack((x1.reshape(-1, 1), 
                     x2.reshape(-1, 1), 
                     x3.reshape(-1, 1),
                #      x4.reshape(-1, 1), 
                     x5.reshape(-1, 1),
                #      x6.reshape(-1, 1),
                     x7.reshape(-1, 1),
                     x8.reshape(-1, 1),
                #      x9.reshape(-1, 1),
                     x10.reshape(-1, 1),
                     x11.reshape(-1, 1),
                     x12.reshape(-1, 1),
                #      x13.reshape(-1, 1),
                     x14.reshape(-1, 1),
                
                     ))

Y = np.column_stack((y1,y2,y3,y4))

RF_Iteration_Scores = {'CV': [], 'R2': [],  'R2_train': []}
MaxGlobal = 0
MaxGlobal_Score = 0
MaxGlobal_Score_train = 0
Max_local = 0
RF_scores = {'CV': [], 'R2': [],  'R2_train': []}
for j in range(50):
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', category=DeprecationWarning)
    
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = j)
    
    
    xscaler = preprocessing.StandardScaler()
    X_train_standard = xscaler.fit_transform(X_train)
    X_test_standard = xscaler.transform(X_test)
    
    
    yscaler = preprocessing.StandardScaler()
    Y_train_standard = yscaler.fit_transform(Y_train)
    Y_test_standard = yscaler.transform(Y_test)
    
    
    rf = RandomForestRegressor()

   
    params_RF = {
        'n_estimators':(2,40),
        'max_features':(2,10),
        'min_samples_split':(2,10),
        'min_samples_leaf':(2,10),
        'max_leaf_nodes':(2,10),
        'bootstrap':[True, False]
    }
    
    total_iter = 20
    #define partitioning
    skf = RepeatedKFold(n_splits=5,random_state=22)
    for baseEstimator in ['GP','RF','ET']:
            opt = BayesSearchCV(estimator = rf,
            search_spaces=params_RF,
            n_iter=total_iter,n_jobs=-1,cv=skf,return_train_score=False,optimizer_kwargs={'base_estimator': baseEstimator})
            report_perf(opt, X_train_standard, Y_train_standard,'BayesSearchCV_'+baseEstimator)
     
    opt.fit(X_train_standard,  Y_train_standard)
    best_model = opt.best_estimator_
    
    # cross validation
    Cross_validation_RF = cross_val_score(best_model, X_train_standard, Y_train_standard, cv=5)
    Cross_validation_Accuracy_RF = Cross_validation_RF.mean().round(2)
    #print('Cross_validation_Accuracy_LR = ',  Cross_validation_Accuracy_RF )
    # evaluation model
    if  Cross_validation_Accuracy_RF > Max_local:
        Max_local = Cross_validation_Accuracy_RF
        Y_pred_RF = best_model.predict(X_test_standard)
        Y_pred_RF_train = best_model.predict(X_train_standard)

        Score_local = r2_score(Y_test_standard, Y_pred_RF)
        Score_local_train = r2_score(Y_train_standard, Y_pred_RF_train)
    
    RF_scores['CV'].append(Max_local)
    RF_scores['R2'].append(Score_local)
    RF_scores['R2_train'].append(Score_local_train)
    
   
    if Max_local > MaxGlobal and Score_local > MaxGlobal_Score:
        MaxGlobal=Max_local
        MaxGlobal_Score= Score_local
        MaxGlobal_Score_train=Score_local_train
        RF_BestFit=best_model
        RF_BestFit_iter = (j)
        print(RF_BestFit_iter)
        RF_X_train_standard=X_train_standard
        RF_Y_train_standard=Y_train_standard
        RF_X_test_standard=X_test_standard
        RF_Y_test_standard=Y_test_standard
        
        RF_Y_train_standard_realscale=yscaler.inverse_transform(Y_train_standard)
        RF_Y_test_standard_realscale=yscaler.inverse_transform(Y_test_standard)
        ypredtrain=RF_BestFit.predict(X_train_standard)
        ypredtest=RF_BestFit.predict(X_test_standard)
        RF_pred_Y_train_standard_realscale=yscaler.inverse_transform(ypredtrain)
        RF_pred_Y_test_standard_realscale=yscaler.inverse_transform(ypredtest)

        




In [ ]:
print(RF_BestFit)

In [ ]:

S = pd.DataFrame(RF_scores)
print(RF_BestFit_iter)

print("CV = ", S.CV)
print("R2 = ", S.R2)
print("R2 train = ", S.R2_train)

print("mean CV {0}".format(S.CV.mean()))
print("mean R2 {0}".format(S.R2.mean()))
print("mean R2_train {0}".format(S.R2_train.mean()))

print("test r2 = {}".format(r2_score(RF_Y_test_standard,RF_pred_Y_test_standard_realscale)))
print("test RMSE: {0} and MAE: {1}".format(sqrt(np.mean((RF_Y_test_standard - ypredtest)**2)),mae(RF_Y_test_standard, ypredtest).round(3)))

print("train r2 = {}".format(r2_score(RF_Y_train_standard_realscale,RF_pred_Y_train_standard_realscale)))
print("train RMSE: {0} and MAE: {1}".format(sqrt(np.mean((RF_Y_train_standard_realscale - RF_pred_Y_train_standard_realscale)**2)),mae(RF_Y_train_standard_realscale, RF_pred_Y_train_standard_realscale).round(3)))
RF_test_r2 = r2_score(RF_Y_test_standard_realscale,RF_pred_Y_test_standard_realscale)
RF_test_RMSE = sqrt(np.mean((RF_Y_test_standard_realscale - RF_pred_Y_test_standard_realscale)**2))
RF_train_r2 = r2_score(RF_Y_train_standard_realscale,RF_pred_Y_train_standard_realscale)
RF_train_RMSE = sqrt(np.mean((RF_Y_train_standard_realscale - RF_pred_Y_train_standard_realscale)**2))

RF_CV_Mean = S.CV.mean()
joblib.dump(RF_BestFit, './model/RF_model.pkl')
joblib.dump(xscaler, './model/RF_Xscaler.pkl')
joblib.dump(yscaler, './model/RF_Yscaler.pkl')
# Save data
np.save('./model_data/RF_Y_train_standard_realscale',RF_Y_train_standard_realscale)
np.save('./model_data/RF_Y_test_standard_realscale',RF_Y_test_standard_realscale)
np.save('./model_data/RF_pred_Y_train_standard_realscale',RF_pred_Y_train_standard_realscale)
np.save('./model_data/RF_pred_Y_test_standard_realscale',RF_pred_Y_test_standard_realscale)
np.save('./model_data/RF_X_train_standard',RF_X_train_standard)
np.save('./model_data/RF_Y_train_standard',RF_Y_train_standard)
np.save('./model_data/RF_X_test_standard',RF_X_test_standard)
np.save('./model_data/RF_Y_test_standard',RF_Y_test_standard)


np.save('./model_data/RF_scores',RF_scores)

In [ ]:
'''
Validation DataSets
'''
file_name = 'Test_Phase.xlsx'
dataset = pd.read_excel(file_name, header=0)
#print(dataset)
x1 = dataset['P(W)'].values
x2 = dataset['V(mm/s)'].values
x3 = dataset['h(um)'].values
# x4 = dataset['t(um)'].values
x5 = dataset['LED(J/mm)'].values
# x6 = dataset['SED(J/mm2)'].values
x7 = dataset['VED(J/mm3)'].values
x8 = dataset['Ni'].values
# x9 = dataset['Ti'].values
x10 = dataset['Dmin'].values
x11 = dataset['Dmax'].values
x12 = dataset['D10'].values
# x13 = dataset['D50'].values
x14 = dataset['D90'].values


y1 = dataset['Ms'].values
y2 = dataset['Mf'].values
y3 = dataset['As'].values
y4 = dataset['Af'].values
X = np.column_stack((x1.reshape(-1, 1), 
                     x2.reshape(-1, 1), 
                     x3.reshape(-1, 1),
                #      x4.reshape(-1, 1), 
                     x5.reshape(-1, 1),
                #      x6.reshape(-1, 1),
                     x7.reshape(-1, 1),
                     x8.reshape(-1, 1),
                #      x9.reshape(-1, 1),
                     x10.reshape(-1, 1),
                     x11.reshape(-1, 1),
                     x12.reshape(-1, 1),
                #      x13.reshape(-1, 1),
                     x14.reshape(-1, 1),
                
                     ))

Y = np.column_stack((y1,y2, y3, y4))


print(Y)

In [ ]:
'''
Validation DataSets
'''
x_standard = xscaler.transform(X)
y_standard = yscaler.transform(Y)
y_pre_standard = RF_BestFit.predict(x_standard)
y_pre_real = yscaler.inverse_transform(y_pre_standard)
print(Y)
print(y_pre_real.round(0))

In [ ]:
'''
All DataSets
'''
file_name = 'NiTi_PhaseDatesets.xlsx'
dataset = pd.read_excel(file_name, header=0)
#print(dataset)
x1 = dataset['P(W)'].values
x2 = dataset['V(mm/s)'].values
x3 = dataset['h(um)'].values
# x4 = dataset['t(um)'].values
x5 = dataset['LED(J/mm)'].values
# x6 = dataset['SED(J/mm2)'].values
x7 = dataset['VED(J/mm3)'].values
x8 = dataset['Ni'].values
# x9 = dataset['Ti'].values
x10 = dataset['Dmin'].values
x11 = dataset['Dmax'].values
x12 = dataset['D10'].values
# x13 = dataset['D50'].values
x14 = dataset['D90'].values

y1 = dataset['Ms'].values
y2 = dataset['Mf'].values
y3 = dataset['As'].values
y4 = dataset['Af'].values
all_X = np.column_stack((x1.reshape(-1, 1), 
                     x2.reshape(-1, 1), 
                     x3.reshape(-1, 1),
                #      x4.reshape(-1, 1), 
                     x5.reshape(-1, 1),
                #      x6.reshape(-1, 1),
                     x7.reshape(-1, 1),
                     x8.reshape(-1, 1),
                #      x9.reshape(-1, 1),
                     x10.reshape(-1, 1),
                     x11.reshape(-1, 1),
                     x12.reshape(-1, 1),
                #      x13.reshape(-1, 1),
                     x14.reshape(-1, 1),
               
                     ))

all_Y = np.column_stack((y1,y2, y3, y4))
# Y = y1.reshape(-1, 1)
all_Y = all_Y.round(0)


print(len(all_Y))

In [ ]:
'''
All DataSets
'''
all_x_standard = xscaler.transform(all_X)
all_y_standard = yscaler.transform(all_Y)
all_y_pre_standard = RF_BestFit.predict(all_x_standard)
all_y_pre_real = yscaler.inverse_transform(all_y_pre_standard)
print(all_Y)
print(all_y_pre_real.round(0))

In [ ]:
'''
All DataSets
'''
all_y_pre_real = all_y_pre_real.round(0)
print("All DataSets r2 = {}".format(r2_score(all_y_standard, all_y_pre_standard)))
print("All DataSets RMSE: {0} and MAE: {1}".format(sqrt(np.mean((all_y_standard - all_y_pre_standard)**2)),mae(all_y_standard, all_y_pre_standard).round(3)))  
